In [2]:
import requests
import pandas as pd

In [2]:
url = 'https://ch.tetr.io/api/users/lists/league/all'

response = requests.get(url)

In [3]:
x = response.json()

In [14]:
x['data']['users'][0]

{'_id': '615fb20fe17beeef45104302',
 'username': 'blaarg',
 'role': 'user',
 'xp': 4757971,
 'league': {'gamesplayed': 1531,
  'gameswon': 1319,
  'rating': 24999.07134059898,
  'glicko': 4181.568585951683,
  'rd': 74.65689571610501,
  'rank': 'x',
  'bestrank': 'x',
  'apm': 176.33,
  'pps': 3.39,
  'vs': 365.08,
  'decaying': False},
 'supporter': True,
 'verified': True,
 'country': 'US'}

In [6]:
rows_list = []
for user in x['data']['users']:
    row_info = {'id': user['_id'],
                'username': user['username'], 
                'gamesplayed': user['league']['gamesplayed'],
                'gameswon': user['league']['gameswon'],
                'tr': user['league']['rating'],
                'glicko': user['league']['glicko'],
                'rd': user['league']['rd'],
                'rank': user['league']['rank'],
                'bestrank': user['league']['bestrank'],
                'apm': user['league']['apm'],
                'pps': user['league']['pps'],
                'vs': user['league']['vs'],
                'verified': user['verified']
                'country': user['country']}
    
    rows_list.append(row_info)
    
df = pd.DataFrame(rows_list)

In [7]:
# Drop unranked players
df = df.drop(df[df['rank'] == 'z'].index)

### 40L / Blitz data

In [60]:
sprints = []
blitzes = []

import time
i = 0
t0 = time.time()

for user in df['username']:
    if i % 1000 == 0:
        print(f"{i} users done, time {time.time()-t0:.2f}")
    url = f'https://ch.tetr.io/api/users/{user}/records'
    response = requests.get(url)
    x = response.json()
    
    if not x['success']:
        continue
    
    try:
        sprint_time = round(x['data']['records']['40l']['record']['endcontext']['finalTime']/1000, 2)
    except TypeError:
        sprint_time = None
        
    try:
        blitz_score = x['data']['records']['blitz']['record']['endcontext']['score']
    except TypeError:
        blitz_score = None
        
    i += 1
        
    sprints.append(sprint_time)
    blitzes.append(blitz_score)

0 users done, time 0.00
1000 users done, time 114.62
2000 users done, time 251.63
3000 users done, time 386.21
4000 users done, time 506.13
5000 users done, time 624.63
6000 users done, time 735.03
7000 users done, time 857.15
8000 users done, time 968.41
9000 users done, time 1085.68
10000 users done, time 1198.22
11000 users done, time 1315.15
12000 users done, time 1428.94
13000 users done, time 1548.81
14000 users done, time 1689.02
15000 users done, time 1814.14
16000 users done, time 1931.85
17000 users done, time 2051.84
18000 users done, time 2173.38
19000 users done, time 2280.04
20000 users done, time 2388.79
21000 users done, time 2498.44
22000 users done, time 2610.21
23000 users done, time 2719.45
24000 users done, time 2820.84
25000 users done, time 2933.07
26000 users done, time 3040.89
27000 users done, time 3171.65
28000 users done, time 3278.55
29000 users done, time 3381.55
30000 users done, time 3495.25
31000 users done, time 3603.96
32000 users done, time 3730.37
3

In [62]:
df['40l_sprint'] = sprints
df['blitz'] = blitzes

In [69]:
df.to_csv('tl-data.csv', index=False)

### Unranked users (from xp leaderboard)

In [5]:
url = 'https://ch.tetr.io/api/users/lists/xp'
response = requests.get(url)
x = response.json()

In [27]:
url = 'https://ch.tetr.io/api/users/lists/xp?after=265000'
response = requests.get(url)
response.json()

{'success': True,
 'data': {'users': []},
 'cache': {'status': 'miss',
  'cached_at': 1668445040606,
  'cached_until': 1668445640606}}